<iframe src="figures/knit11.pdf" width="600" height="400"></iframe>


<iframe src="figures/knit11.pdf" width="600" height="400"></iframe>




# Real-Time DDoS Detection with Amlight Network Data and P4 Switches

In this experiment, we demonstrate real-time DDoS detection using Amlight network data and P4 switches. The setup includes a facility port link to FIU, where we connect to a live data feed.

The P4 switch maintains registers to track packet counts, as well as source and destination IP addresses. These register values are processed at the control plane to identify IP addresses with the highest packet counts and the most unique source IPs. Based on this ranking, the controller issues a P4Runtime write request to install a rule in the data plane for the selected IP address.

The data plane then generates a digest containing header information for this entry. The controller receives the digest, aggregates statistics such as the average and standard deviation, and feeds the results into a machine learning model. The model produces predictions and classifies the traffic as either normal flow or attack flow.

This tutorial is designed to help researchers learn how to leverage Amlight resources through the FABRIC testbed and to experiment with P4 code for advanced network research.

The experimental setup is shown in the figure below:

<img src="figures/knit11.png" alt="Figure description" width="800"/>


## 1. Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager(project_id = "1ecd9d6a-7701-40fa-b78e-b2293c9526ed")

fablib.show_config();

# 2. Create the Experiment Slice

The following script sets up a node and facility port, each connected to two ports on a P4 tofino switch.


### 2A. Identify the Facility port and Node Sites

List the available sites with a `P4 Switch`. Select one site at random from the available options. In this experiment, we choose to allocate all resources at FIU.

In [ ]:
p4_column_name = "p4-switch_available"

'''
# Find a site which has a P4 Switch available
[site2] = fablib.get_random_sites(count=1, filter_function=lambda x: x[p4_column_name] > 0)

# Choose another random site other than P4 site to host the VMs
site1 = fablib.get_random_site(avoid=[site2])
'''

[site2] = fablib.get_random_sites(count=1, filter_function=lambda x: x[p4_column_name] > 0)
site1 = fablib.get_random_site(avoid=[site2])

facility_port_site=site1

print(f"Sites chosen for hosting VMs: {site1} P4: {site2} facility port: {facility_port_site}")

In [ ]:
p4_column_name = "p4-switch_available"

site1 = "FIU"
site2 = "FIU"

facility_port_site="FIU"

print(f"Sites chosen for hosting VMs: {site1} P4: {site2} facility port: {facility_port_site}")

### 2B. Identify the VLAN for the Facility Port
The script below displays the VLANs available at the FIU site.

In [ ]:
cell_output = 'pandas'
output_list = fablib.list_facility_ports(filter_function=lambda x: x['site_name'] == facility_port_site, output=cell_output)

In [ ]:
# Choose one of the available VLANs
facility_port_vlan='4015'

### 2C. Define the variable names 

In [ ]:
# define the variables
facility_port_name = "AmLight-EXP-Layer2-FIU"
slice_name = "DDoS with p4 - 1"
node1_name = "DestNode"
p4_name = "P4"
network1_name = "net1"
network2_name = "net2"
model = "NIC_Basic"

### 2D. Create the Slice 
This step submits a request to create a slice for the experiment with the specified devices and parameters. The process may take 5–7 minutes to complete. Once finished, it will display the details of the slice.

The output table also shows how to SSH into the P4 switch and nodes. From there, you can run the experiments directly from the terminal if desired.

In [ ]:
# Create Slice
slice = fablib.new_slice(name=slice_name)

# Create Node 1 and its links
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model=model, name="nic1").get_interfaces()[0]
# iface1.set_mode("config")

# Optional: Uncomment if Node1 needs an IP address
# iface1.set_ip_addr(IPv4Address("192.168.0.2"))

# Create P4 switch and its links
p4 = slice.add_switch(name=p4_name, site=site2)
iface2 = p4.get_interfaces()[0]  # Connects to net1
iface3 = p4.get_interfaces()[1]  # Connects to net2


# Set VLAN tag on P4 switch's interface (iface3) to match facility port
# iface3.set_vlan(facility_port_vlan)  # Sets VLAN to '4018' (string)

# Add Facility Port
facility_port = slice.add_facility_port(name=facility_port_name, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface = facility_port.get_interfaces()[0]


# Create Networks
net1 = slice.add_l2network(name=network1_name, interfaces=[])
net1.add_interface(iface1)
net1.add_interface(iface2)
net2 = slice.add_l2network(name=network2_name, interfaces=[])  # Layer 2 network for facility port
net2.add_interface(iface3)
net2.add_interface(facility_port_interface)

# Submit Slice Request
try:
    slice.submit()
except Exception as e:
    print(f"Slice submission failed: {e}")

### 2E. Configure the internet connection

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=f"DestNode")        
node1_iface = node1.get_interface(network_name=f'net1') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet= subnet)

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')

The above `ping` should not work because the L2VPN has not been established and the P4 switch is not yet configured. In the next step, we will set up the Amlight SDX and the FABRIC connection.

## 3. Setup of Amlight AtlanticWave-SDX Connection and L2VPN Creation

## 3A. Check if sdxclient is installed, uninstall it if needed

In [ ]:
!pip show sdxclient
#!pip uninstall -y sdxlib  # Uninstall if installed

In [ ]:
!pip install sdxclient

## 3B. Instantiate the SDX client

In [ ]:
from IPython.display import HTML
from sdxclient.client import SDXClient
client = SDXClient()

### 3C. Check allocated l2VPNs

In [ ]:
client.get_l2vpns()

### 3D. Check available ports

In [ ]:
HTML(client.get_available_ports(format="html", limit=20)["data"])

### 3E. Filter Entities

In [ ]:
HTML(client.get_available_ports(format="html", fields="Domain,Device,Port,Status,Port ID,Entities", limit=20)["data"])

### 3F. Get vlans in use

In [ ]:
HTML(client.get_available_ports(format="html", fields="Domain,Device,Port,Status,Port ID,Entities,VLANs in Use", limit=20)["data"])

### 3G. Check VLAN availability
From the table above, the port with the entity name `FABRIC FIU – Miami` corresponds to the Fabric testbed side. For live telemetry, select the port with the entity name `Telemetry Feed`. The VLAN for the Fabric node is already chosen when the slice is created. However, you should verify which VLANs are available on the live feed port.

In [ ]:
HTML(client.get_available_ports(format="html", fields="Domain,Device,Port,Status,Port ID,Entities,VLANs Available", limit=20)["data"])

## 3H. Set L2VPN parameters

### SET ENDPOINTS

In [ ]:
# Assign the available ports and VLANs to the fabric port and the FIU server (facility port).
client.set_endpoint(endpoint_position="first", filter="SW17:7", vlan=4015)
client.set_endpoint(endpoint_position="second", filter="SW17:27", vlan=3000)

### SET the L2VPN payload

In [ ]:
client.set_l2vpn_payload(name="Live Telemetry", notifications=[{"email":"lmarinve@fiu.edu"}])

## 3I. Create an L2VPN for live feed

In [ ]:
client.create_l2vpn_from_selection()

### 3J. Fetch the created L2VPN by service_id

In [ ]:
client.get_l2vpn()

## 3K. Check internet connection

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')

## 4. Compile the P4 Script, Load It onto the Switch, and Configure the Switch

**This section and the next can be completed by SSH-ing into the P4 switch and running the experiments from the command line there.** We recommend this approach.
However, it can also be completed using the provided Jupyter notebooks.

If you chose to use the terminal, Follow these steps:

- Run 4A and 4B in one terminal.

- Open another terminal and run 4C there.

- For the experiments, use the second terminal or open a new one.

- Instead of running 5B–5E individually, use the provided script `cNdAmlight_menu.py` that combines all steps of the experiment.


```python
python3 DDoS_Detection_wP4/menu/cNdAmlight_menu.py
```

In [ ]:
# Upload the P4 Code to the switch

import os

print(os.getcwd())

slice = fablib.get_slice(slice_name)
node1 = slice.get_node("DestNode")
switch = slice.get_node("P4")
# switch.upload_directory("DDoS_Detection_wP4", ".")


### 4A. Compile the basic P4 program

#### Configure the environment variables

SDE environment can be setup using the command:

```
sde-env-9.13.3
```

Example output is shown below:
```
fabric@P4:~$ sde-env-9.13.3

Intel Tofino SDE 9.13.3 on platform "accton_wedge100bf_32x"
Load/unload kernel modules: $ sudo $(type -p bf_{kdrv,kpkt,knet}_mod_{load,unload})

Compile: $ p4_build.sh <p4name>.p4
Run:     $ run_switchd.sh -p <p4name>

Build artifacts and logs are stored in /home/fabric/.bf-sde/9.13.3

Use "exit" or CTRL-D to exit this shell.
```
#### Compile the code

In this example we will not modify the P4 code. Instead, we will just compile it and run it on the switch. 

The script `p4_build.sh` is provided by Intel’s support portal. It invokes the compiler to generate the output files. It automatically detects the P4 version (i.e., P416), and generates
the output files under `~/.bf-sde/9.13.3/build/<program_name>`, where `<program_name>` is the file `cNdAmlight.p4`. It also generates the log files under `~/.bf-sde/9.13.3/logs/<program_name>` and other files (e.g., graphs).

Use the command below to compile the code:

```
p4_build.sh ~/DDoS_Detection_wP4/p4src/cNdAmlight.p4
```

After executing the command, if there are no error messages displayed in the terminal, then the P4 program was compiled successfully.

In [ ]:
# Compile the p4 program 

stdout, stderr = switch.execute(command=[

    # Step 1: Enter the Intel Tofino SDE environment
    # This command activates the environment needed to run P4-related tools.
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10),

    # Step 2: Compile the P4 program using p4_build.sh
    # This builds the P4 program specified in the path (basic.p4).
    # The build process may take some time, so a timeout of 20 seconds is used.
    ("p4_build.sh ~/DDoS_Detection_wP4/p4src/cNdAmlight.p4", r"\[nix\-shell\(SDE-9.13.3\):.*\$ ", 20),

    # Step 3: Exit the SDE environment cleanly
    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE-9.13.3\):.*\$ ", 10)
])

# stdout: Captures the standard output of the executed commands.
# stderr: Captures any error messages encountered during execution.


### Verify the compilation output files
Verify the output files for the program basic.p4 are generated in the `~/.bf-sde/9.13.3/build/<program_name>` directory by issuing the following command.

```
ls ~/.bf-sde/9.13.3/build/cNdAmlight.p4/
```

The binary file that corresponds to the compiled data plane is located under `ls ~/.bf-sde/9.13.3/build/<program_name>/tofino/pipe`. Use the command below to display the contents of this directory.

```
ls ~/.bf-sde/9.13.3/build/cNdAmlight.p4/tofino/pipe/
```

In [ ]:
stdout, stderr = switch.execute("ls ~/.bf-sde/9.13.3/build/cNdAmlight/")

In [ ]:
stdout, stderr = switch.execute("ls ~/.bf-sde/9.13.3/build/cNdAmlight/tofino/pipe/")

### 4B. Start the switch daemon and add the ports 

Now that we have compiled our P4 program and generated the output files. We can start the switch daemon with the compiled output using the command below.


Load the  kernel module, and start the switch daemon using the following commands:
```
sde-env-9.13.3

sudo $SDE_INSTALL/bin/./bf_kdrv_mod_load $SDE_INSTALL

run_switchd.sh -p cNdAmlight
```

Once the daemon is running, you can issue commands in the bfshell.  

1. Issue the following command in the switch CLI to manage the ports of the switch.

```
ucli
```

NOTE: The ucli is the bfshell instance used to manage the switch ports. With the ucli, you can enable or disable ports, set the port speed (e.g., 100 Gbps, 40 Gbps, and 10 Gbps), and
select the FEC type. Additionally, the user can monitor the status of the ports, the number of sent and received frames, and other variables.

2. Now we will add the ports for the switch. Recall from the topology that Server1 is connected to port 1 on the switch, and Server2 is connected to port 2 on the switch. Issue the
commands below to add ports 1 and 2 in the Tofino switch.

```
pm port-add 1/- 100G NONE
pm port-add 2/- 100G NONE
```

3. Enable the ports by issuing the following commands.
```
pm port-enb 1/-
pm port-enb 2/-
```

4. Verify that the ports are up by issuing the following command.
```
pm show
```


**Expected output:**
```
bf-shell> ucli
bf-sde> pm port-add 1/- 100G NONE
bf-sde> pm port-add 2/- 100G NONE
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|132|3/ 4|100G   |NONE|Au|Au|YES|ENB|DWN|  NONE  |               0|               0|
2/0  |22/0|140|3/12|100G   |NONE|Au|Au|YES|ENB|DWN|  NONE  |               0|               0|
bf-sde> pm port-enb 1/-
bf-sde> pm port-enb 2/-
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|132|3/ 4|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |              29|              14|
2/0  |22/0|140|3/12|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |              29|              14|
```

The ouput above shows that ports 1 and 2 are up. The speed for the ports is 100G. The columns `FRAMES TX` indicate how many frames are received and `FRAMES RX` transmitted in each port respectively.

**NOTE**: These commands can also be executed manually by accessing the P4 switch via SSH.

In [ ]:
switch = slice.get_node("P4")
thread = switch.execute_thread(command=[
    # Enter SDE environment
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10),

    # Load Kernel Modules
    ("sudo $SDE_INSTALL/bin/./bf_kdrv_mod_load $SDE_INSTALL", r"\[nix\-shell\(SDE-9.13.3\):.*\$ ", 20),

    # Start run_switchd.sh interactively (DO NOT run in background)
    ("run_switchd.sh -p cNdAmlight", r"bfshell>", 30),  # Wait for switchd prompt to appear

    # Enter UCLI after switchd starts
    ("ucli", r"bf-sde>", 10),

    # Port configuration inside UCLI
    ("pm port-add 1/- 100G NONE", r"bf-sde>", 10),
    ("pm port-add 2/- 100G NONE", r"bf-sde>", 10),
    ("pm port-enb 1/-", r"bf-sde>", 10),
    ("pm port-enb 2/-", r"bf-sde>", 10),
    ("pm show", r"bf-sde>", 10),
    # Keep the session open to prevent exit
    ("sleep infinity", r"bf-sde>", 3000) 
], output_file="run_switchd.log")

### 4C. Populating the switch’s forwarding table

**1. Update the port numbers in the script excerpt below to match the D_P column from the pm show output above.**

```python
forwarding.add_with_send_using_port(ingress_port=132, egress_port=140)
forwarding.add_with_send_using_port(ingress_port=140, egress_port=132)
```


**NOTE**: These steps can also be executed manually by accessing the P4 switch via SSH.

In [ ]:
file_path = "~/DDoS_Detection_wP4/bfrt_python/cNdAmlight_setup.py"
INGRESS = switch.get_interfaces()[0].get_device_name()
EGRESS = switch.get_interfaces()[1].get_device_name()

print(INGRESS)
print(EGRESS)

switch.execute(
    f"sed -i 's/ingress_logical *= *[0-9]\\+/ingress_logical = {INGRESS}/' {file_path}"
)
switch.execute(
    f"sed -i 's/egress_logical *= *[0-9]\\+/egress_logical = {EGRESS}/' {file_path}"
)


**2. Populate the table entries to the switch by typing the following command in the embedded terminal.**

```
sde-env-9.13.3

run_bfshell.sh --no-status-srv -b ~/DDoS_Detection_wP4/bfrt_python/cNdAmlight.py
```

The forwarding table contains the information that the P4 program will use to forward packets to the right destination. 

**NOTE**: Please wait to run this step until you see ports as shown below to ensure the port have been enabled by the previous step. If you see an error, please re-run this step.

**NOTE**: These steps can also be executed manually by accessing the P4 switch via SSH.

```
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|132|2/ 4|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |               7|               0| 
2/0  |22/0|140|2/12|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |               7|               0|
```

In [ ]:
# Define the port for the forwarding table
stdout, stderr = switch.execute(command=[
    
    # Step 1: Enter the Intel Tofino SDE environment
    # This command activates the environment needed to run P4-related tools.
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10),

    # Step 2: Run the BFShell script to set up the P4 runtime environment
    # This script initializes the environment and configures the necessary settings.
    # --no-status-srv: Disables the status server.
    # -b <script>: Runs the Python setup script inside bfshell.
    ("run_bfshell.sh --no-status-srv -b ~/DDoS_Detection_wP4/bfrt_python/cNdAmlight_setup.py", 
     r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 30),

    # Step 3: Exit the SDE environment cleanly
    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])

# stdout will contain the standard output from the executed commands.
# stderr will contain any error messages encountered during execution.


### 4D. Verify Reachability through Ping Test

Initiate a ping between the VMs.  

**NOTE:** The ping will function as long as the switch daemon is actively running on the Tofino switch and is correctly configured.  

In this example, the switch daemon automatically terminates after **1 hour**, which may cause the ping to stop working beyond this duration. This is expected behavior. To resume pinging, the user must restart the switch daemon.

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')

## 5. Run DDoS Experiment

### 5A. Installing required libraries

In [ ]:
stdout, stderr = switch.execute(command=[
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 15),
    ("unset PYTHONPATH LD_LIBRARY_PATH", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),

    # Nix libs only (avoid /usr/lib to prevent GLIBC mismatch)
    ("GCC_LIB=$(ls -d /nix/store/*-gcc-*-lib/lib 2>/dev/null | head -n1 || true)", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),
    ("ZLIB_SO=$(find /nix/store -maxdepth 3 -type f -name 'libz.so.1' 2>/dev/null | head -n1 || true)", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),
    ("[ -n \"$ZLIB_SO\" ] && ZLIB_LIB=$(dirname \"$ZLIB_SO\") || ZLIB_LIB=''", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),
    ("[ -z \"$ZLIB_LIB\" ] && [ -e /lib/x86_64-linux-gnu/libz.so.1 ] && mkdir -p ~/shimlibs && cp -f /lib/x86_64-linux-gnu/libz.so.1 ~/shimlibs/ && ZLIB_LIB=~/shimlibs || true", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 15),
    ("[ -n \"$GCC_LIB\" ] && export LD_LIBRARY_PATH=\"$GCC_LIB\" || true", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),
    ("[ -n \"$ZLIB_LIB\" ] && export LD_LIBRARY_PATH=\"$ZLIB_LIB:$LD_LIBRARY_PATH\" || true", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),

    ("VENV=~/sde_venv", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 10),
    ("rm -rf \"$VENV\"", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 15),
    ("python3 -m venv \"$VENV\"", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 15),
    ("source \"$VENV/bin/activate\"", r"\(sde_venv\).*\$ ", 10),

    # Quiet pip: no version check, no progress bar, -q
    ("export PIP_DISABLE_PIP_VERSION_CHECK=1 PIP_PROGRESS_BAR=off", r"\(sde_venv\).*\$ ", 10),
    ("pip install -q --no-cache-dir --upgrade pip", r"\(sde_venv\).*\$ ", 20),
    ("pip install -q --no-cache-dir numpy pandas grpcio joblib scikit-learn", r"\(sde_venv\).*\$ ", 100),

    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])


### 5B. Clear the registers

In [ ]:
# Run the command below  
# To clear the registers and start again over 

stdout, stderr = switch.execute(command=[
    # 1) Enter SDE and activate venv
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 20),
    ("source ~/sde_venv/bin/activate", r"\(sde_venv\).*\$ ", 10),

    # 2) Clean Python path; set only the two Nix lib folders if they exist (no typos, no /usr/lib)
    ("unset PYTHONPATH LD_LIBRARY_PATH; "
     "for p in /nix/store/*-gcc-*-lib/lib /nix/store/*-zlib-*/lib; do "
     "  [ -d \"$p\" ] && LD_LIBRARY_PATH=\"$p:${LD_LIBRARY_PATH}\"; "
     "done; export LD_LIBRARY_PATH", r"\(sde_venv\).*\$ ", 10),

    # 3) Run your script, quieter logging (without changing the file)
    ("cd /home/fabric && PYTHONWARNINGS=ignore "
     "python -c \"import logging,runpy; logging.getLogger().setLevel(logging.ERROR); "
     "runpy.run_path('DDoS_Detection_wP4/menu/clear_registers.py', run_name='__main__')\"",
     r"\(sde_venv\).*\$ ", 20),

    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])

### 5C. Inspect the registers

In [ ]:
# Run the command below  
# To inspect the registers  
# If there are non-zero entries, the output will show the top and bottom 15 IP addresses by packet count  

stdout, stderr = switch.execute(command=[
    # 1) Enter SDE and activate venv
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 20),
    ("source ~/sde_venv/bin/activate", r"\(sde_venv\).*\$ ", 10),

    # 2) Clean Python path; set only the two Nix lib folders if they exist (no typos, no /usr/lib)
    ("unset PYTHONPATH LD_LIBRARY_PATH; "
     "for p in /nix/store/*-gcc-*-lib/lib /nix/store/*-zlib-*/lib; do "
     "  [ -d \"$p\" ] && LD_LIBRARY_PATH=\"$p:${LD_LIBRARY_PATH}\"; "
     "done; export LD_LIBRARY_PATH", r"\(sde_venv\).*\$ ", 10),

    # 3) Run your script, quieter logging (without changing the file)
    ("cd /home/fabric && PYTHONWARNINGS=ignore "
     "python -c \"import logging,runpy; logging.getLogger().setLevel(logging.ERROR); "
     "runpy.run_path('DDoS_Detection_wP4/menu/inspect_registers.py', run_name='__main__')\"",
     r"\(sde_venv\).*\$ ", 40),

    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])

### 5D. Add digest entries and request digest messages

In [ ]:
# Run the command below  
# To add digest entries and create digest messages for those entries

stdout, stderr = switch.execute(command=[
    # 1) Enter SDE and activate venv
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 20),
    ("source ~/sde_venv/bin/activate", r"\(sde_venv\).*\$ ", 10),

    # 2) Clean up env and set LD_LIBRARY_PATH
    ("unset PYTHONPATH LD_LIBRARY_PATH; "
     "for p in /nix/store/*-gcc-*-lib/lib /nix/store/*-zlib-*/lib; do "
     "  [ -d \"$p\" ] && LD_LIBRARY_PATH=\"$p:${LD_LIBRARY_PATH}\"; "
     "done; export LD_LIBRARY_PATH", r"\(sde_venv\).*\$ ", 10),

    # 3) Run your script with args
    ("cd /home/fabric && PYTHONWARNINGS=ignore "
     "python DDoS_Detection_wP4/menu/add_entry_and_retrieve_digests.py 146.88.10.70",
     r"\(sde_venv\).*\$ ", 50),

    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])


### 5E. Show aggregated digest entries and ML predictions

In [ ]:
# Run the command below  
# To get the aggreagated digests and DDoS predictions

stdout, stderr = switch.execute(command=[
    # 1) Enter SDE and activate venv
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 20),
    ("source ~/sde_venv/bin/activate", r"\(sde_venv\).*\$ ", 10),

    # 2) Clean up env and set LD_LIBRARY_PATH
    ("unset PYTHONPATH LD_LIBRARY_PATH; "
     "for p in /nix/store/*-gcc-*-lib/lib /nix/store/*-zlib-*/lib; do "
     "  [ -d \"$p\" ] && LD_LIBRARY_PATH=\"$p:${LD_LIBRARY_PATH}\"; "
     "done; export LD_LIBRARY_PATH", r"\(sde_venv\).*\$ ", 10),

    # 3) Run your script 
    ("cd /home/fabric && PYTHONWARNINGS=ignore "
     "python DDoS_Detection_wP4/menu/show_digest_outcomes.py",
     r"\(sde_venv\).*\$ ", 20),

    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])

### 5F. Delete digest entries

In [ ]:
# Run the command below  
# To delete digest entries

stdout, stderr = switch.execute(command=[
    # 1) Enter SDE and activate venv
    ("sde-env-9.13.3", r"\[nix\-shell\(SDE\-9\.13\.3\):.*\$ ", 20),
    ("source ~/sde_venv/bin/activate", r"\(sde_venv\).*\$ ", 10),

    # 2) Clean up env and set LD_LIBRARY_PATH
    ("unset PYTHONPATH LD_LIBRARY_PATH; "
     "for p in /nix/store/*-gcc-*-lib/lib /nix/store/*-zlib-*/lib; do "
     "  [ -d \"$p\" ] && LD_LIBRARY_PATH=\"$p:${LD_LIBRARY_PATH}\"; "
     "done; export LD_LIBRARY_PATH", r"\(sde_venv\).*\$ ", 10),

    # 3) Run your script with args
    ("cd /home/fabric && PYTHONWARNINGS=ignore "
     "python DDoS_Detection_wP4/menu/delete_digest_entries.py",
     r"\(sde_venv\).*\$ ", 20),

    # Ensures that the environment is properly terminated after execution.
    ("exit", r"\[nix\-shell\(SDE\-9.13.3\):.*\$ ", 10)
])

## Delete the L2VPN

In [ ]:
# check all L2VPNs
client.get_l2vpns()

In [ ]:
# Check the L2VPN we created above
client.get_l2vpn()

In [ ]:
# Delete the L2VPN
client.delete_l2vpn()

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
# slice=fablib.get_slice(slice_name)
# slice.delete()